In [42]:
from skimage import io, img_as_float
import itertools
import numpy as np
import random

def load_imgs(set_path):
    imgs = {}
    file = open(set_path + "correct.txt", mode="r")
    lines = file.readlines()
    for i in range(0,len(lines)):
        img = io.imread(set_path + str(i) + ".png")
        img = img_as_float(img)
        imgs[i] = img
    return imgs

    
def score_similarity(pair):
    #TODO compare by better classifier
    img1 = imgs[pair[0]]
    img2 = imgs[pair[1]]
    img1_nz = np.count_nonzero(img1)
    img2_nz = np.count_nonzero(img2)

    return img1_nz - img2_nz

def load_correct_pairs(set_path):
    result = []
    file = open(set_path + "correct.txt", mode="r")
    lines = file.readlines()
    for idx, line in enumerate(lines):
        result.append((idx, int(line)))
    return result


def rank_pairs(pairs):
    scores = []
    for pair in pairs:
        s = score_similarity(pair)
        scores.append((pair, s))
    return sorted(scores, key=lambda x: -x[1])


def score_rank(rank, correct_pair):
    for idx, elem in enumerate(rank):
        if elem[0] == correct_pair:
            return 1/(idx+1)
    return 0
correct_pairs = load_correct_pairs("data/set0/")
imgs = load_imgs("data/set0/")
for correct_pair in correct_pairs:
    queried = correct_pair[0]
    candidates = np.delete(np.arange(0,len(correct_pairs)), queried)
    possible_pairs = list(itertools.product([queried],candidates))
    rank = rank_pairs(possible_pairs)
    print("rank = {} score = {} correct = {}".format(rank, score_rank(rank,correct_pair), correct_pair))




rank = [((0, 3), 16950), ((0, 5), 15474), ((0, 2), 9509), ((0, 4), 9092), ((0, 1), -24529)] score = 0.5 correct = (0, 5)
rank = [((1, 3), 41479), ((1, 5), 40003), ((1, 2), 34038), ((1, 4), 33621), ((1, 0), 24529)] score = 0.3333333333333333 correct = (1, 2)
rank = [((2, 3), 7441), ((2, 5), 5965), ((2, 4), -417), ((2, 0), -9509), ((2, 1), -34038)] score = 0.2 correct = (2, 1)
rank = [((3, 5), -1476), ((3, 2), -7441), ((3, 4), -7858), ((3, 0), -16950), ((3, 1), -41479)] score = 0.3333333333333333 correct = (3, 4)
rank = [((4, 3), 7858), ((4, 5), 6382), ((4, 2), 417), ((4, 0), -9092), ((4, 1), -33621)] score = 1.0 correct = (4, 3)
rank = [((5, 3), 1476), ((5, 2), -5965), ((5, 4), -6382), ((5, 0), -15474), ((5, 1), -40003)] score = 0.25 correct = (5, 0)
